In [ ]:
# %%capture
!pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2 langchain_community

# Agents

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate


# Load OpenAI's LLMs with LangChain
os.environ["OPENAI_API_KEY"] = ""
openai_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0,max_tokens=200)

AttributeError: module 'openai' has no attribute 'DefaultHttpxClient'

In [ ]:
# Create the ReAct template
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [ ]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# Prepare tools
tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent
agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [ ]:
# What is the Price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)



> Entering new AgentExecutor chain...
To find the current price of a MacBook Pro in USD, I will perform a web search. After that, I can calculate the cost in EUR using the provided exchange rate of 0.85 EUR for 1 USD.

Action: duckduck
Action Input: "current price of MacBook Pro in USD"snippet: The 14-inch M4 MacBook Pro with 16GB RAM and 1TB comes with $316 off, bringing the price down from $1,799 to $1,483, the best price we've seen to date., title: This Week's Best Apple Deals: M4 MacBook Pro $509 Off, M3 iMac $400 Off ..., link: https://www.macobserver.com/deals/this-weeks-best-apple-deals-m4-macbook-pro-509-off-m3-imac-400-off-and-more/, snippet: Apple's 2024 MacBook Pro 16-inch can be equipped with an M4 Pro or M4 Max chip. Retail prices start at $2,499 USD, but every configuration is on sale, with the best prices in this guide knocking hundred of dollars off the laptop of your choosing. You can also browse a selection of the top 16-inch MacBook Pro deals in our dedicated round

{'input': 'What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?',
 'output': 'The current price of a MacBook Pro (14-inch) is $1,483, which is approximately €1,260.55.'}

### ReAct Demo Code 2

In [ ]:
%%capture
# update or install the necessary libraries
!pip install --upgrade langchain
!pip install --upgrade python-dotenv
!pip install google-search-results


# import libraries
import openai
import os
from langchain.llms import OpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from dotenv import load_dotenv
load_dotenv()

# load API keys; you will need to obtain these if you haven't yet
os.environ["OPENAI_API_KEY"] = ""
os.environ["SERPER_API_KEY"] = ""

In [ ]:
!pip install openai==0.27.8

from langchain.chat_models import ChatOpenAI


llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0,max_tokens=200)
tools = load_tools(["google-serper", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

/tmp/ipython-input-2-192766835.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0,max_tokens=200)
/tmp/ipython-input-2-192766835.py:8: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/ho

In [ ]:
# run the agent
agent.run("Who are Dr. Raj Dandekar's co-founders? What is their current age raised to the 1 power?")

/tmp/ipython-input-3-2843818337.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("Who are Dr. Raj Dandekar's co-founders? What is their current age raised to the 1 power?")




> Entering new AgentExecutor chain...
To answer the question about Dr. Raj Dandekar's co-founders, I will need to search for information regarding him and his co-founders. After that, I can calculate their current age raised to the 1 power, which will simply be their current age since any number raised to the power of 1 is itself. 

Action: google_serper  
Action Input: "Dr. Raj Dandekar co-founders"  
Observation: Dr. Raj Abhijit Dandekar. MIT PhD, IIT Madras BTech. Co-founder, Vizuara AI Labs ; Dr. Rajat Dandekar​. Purdue PhD, IIT Madras BTech Co-founder, Vizuara AI Labs ... I am currently the co-founder of two startups Vizuara and Videsh. At Vizuara, we are on a… · Experience: Vizuara · Education: Purdue University · Location: ... As the Co-Founder of Vizuara and Videsh, Raj is responsible for spearheading the strategic vision, product development, and growth initiatives of these dynamic ... Industry Presentation: Raj Dandekar, Co Founder, Vizuara · Comments. Missing: Dr. | Show r

"I am unable to provide the ages of Dr. Raj Dandekar's co-founders or their ages raised to the 1 power, as this information is not readily available."